# Peta Timeframe untuk Strategi EMA112, VWAP, dan ML

Notebook ini merangkum bagaimana strategi EMA112 (trend-following), VWAP (mean-reversion),
dan varian sinyal ML dievaluasi di beberapa timeframe. Gunakan tabel di bawah sebagai titik
awal memilih notebook yang sesuai serta mengingatkan lokasi data/output masing-masing.

## Ringkasan Notebook Per Timeframe

| Notebook | Timeframe | Dataset | Folder Output | Catatan |
| --- | --- | --- | --- | --- |
| `strategy_comparison_tf1h.ipyb` | 1H | `data/BINANCE_ETHUSDT.P, 60.csv` | `outputs/strategy_comparison_tf1h/` | Perbandingan EMA112 vs VWAP dengan opsi sinyal ML; fokus volatilitas intraday. |
| `strategy_comparison_tf1d.ipynb` | 1D | `data/BINANCE_ETHUSD, 1D.csv` | `outputs/strategy_comparison_tf1d/` | Versi daily untuk melihat apakah tren/mean-reversion tetap konsisten di time horizon lebih panjang. |
| (Notebook ini) `strategy_timeframe_matrix.ipynb` | N/A | — | — | Dokumentasi singkat mengenai cakupan timeframe dan strategi. |

Semua notebook tetap kompatibel dengan sinyal ML eksternal melalui `ML_PREDICTION_PATHS` sehingga
hasil model dapat diuji berdampingan dengan strategi rule-based.

## Cara Memperluas Uji Timeframe

1. Duplikasi salah satu notebook di atas dan ganti `DATA_PATH`, `OUTPUT_DIR`, serta `output_prefix`
   sesuai pasangan simbol/timeframe baru.
2. Perbarui nilai `BARS_PER_DAY` dan `BARS_PER_YEAR` agar perhitungan CAGR/volatilitas tahunan
   tetap akurat.
3. Simpan hasil backtest ke folder unik supaya artefak antar time frame tidak bercampur.
4. Ekspor seluruh tabel ke Excel (lihat blok `export_tables_to_excel`) untuk memudahkan komparasi
   lintas timeframe di satu file gabungan.